In [33]:
from groundingdino.util.inference import load_model
from ..UniPose import inference_on_a_image as unipose

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Global variables
UNIPOSE_MODEL = unipose.load_model("../UniPose/config_model/UniPose_SwinT.py", "../UniPose/weights/unipose_swint.pth", cpu_only=False)
DINO_MODEL = load_model("groundingdino/config/GroundingDINO_SwinT_OGC.py", "weights/groundingdino_swint_ogc.pth")
UNIPOSE_CONFIG = {'box_threshold': 0.1, 'iou_threshold': 0.9}

OUTPUT_PATH = "../coco_dataset/test_grounding_dino"
INPUT_PATH = "../coco_dataset/imgs_a"
TEXT_PROMPT = "character . "
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25


ImportError: attempted relative import with no known parent package

In [9]:
# List and load images

import os
from glob import glob

def list_images(dir_path):
    # List all files in directory recursively
    return [y for x in os.walk(dir_path) for y in glob(os.path.join(x[0], '*.jpg'))] + \
           [y for x in os.walk(dir_path) for y in glob(os.path.join(x[0], '*.jpeg'))] + \
           [y for x in os.walk(dir_path) for y in glob(os.path.join(x[0], '*.png'))]

from groundingdino.util.inference import load_image

In [10]:
from groundingdino.util.inference import predict

# use grounding dino to extract boxes coordinates
def detect_character(image):
    boxes, logits, phrases = predict(
        model=DINO_MODEL,
        image=image,
        caption=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )
    return boxes, logits, phrases



In [22]:
import torch
from torchvision.ops.boxes import box_convert
import numpy as np

def process_boxes(boxes, image_source):
    h, w, _ = image_source.shape
    boxes = boxes * torch.Tensor([w, h, w, h])
    xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").numpy()
    xyxy = [list(map(int, sublist)) for sublist in xyxy.tolist()]
    return xyxy

In [30]:
import cv2
def crop_box(image, box):
    cropped = image[box[1]:box[3], box[0]:box[2], :]
    return cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)

def crop_boxes(image, boxes):
    return [crop_box(image, box) for box in boxes]

In [ ]:
from ..UniPose import unipose_to_json as unipose_json


def predict_unipose(image):
    keypoints, _ = unipose_json.detect_keypoints(image, UNIPOSE_MODEL, UNIPOSE_CONFIG)
    keypoints, _, _ = unipose_json.process_keypoints(keypoints, )






In [31]:
from groundingdino.util.inference import annotate
import cv2

image_source, image = load_image(".asset/cat_dog.jpeg")
boxes, logits, phrases = detect_character(image)
box1 = process_boxes(boxes, image_source)[0]
box2 = process_boxes(boxes, image_source)[1]
img1 = crop_box(image_source, box1)
img2 = crop_box(image_source, box2)
cv2.imwrite("box1.jpg", img1)

True